# Columns

- JSON
- Server Default
- DateTime
- IN
- String Concat

In [1]:
import sqlalchemy as sa
from utils import *

In [2]:
metadata_obj = sa.MetaData()
Products = sa.Table('products', metadata_obj, 
                sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
                sa.Column('name', sa.VARCHAR(255), default=None, nullable=True),
                sa.Column('meta', sa.JSON, default=None, nullable=True),
                sa.Column('extra', sa.TEXT, default=None, nullable=True)
    )


In [3]:
engine = sa.create_engine('sqlite://')
con = engine.connect()

In [4]:
metadata_obj.create_all(con)
if con.in_transaction():
    con.commit()

# JSON



    Comparator.as_string() - return the element as a string

    Comparator.as_boolean() - return the element as a boolean

    Comparator.as_float() - return the element as a float

    Comparator.as_integer() - return the element as an integer


In [5]:
with con.begin() as t:
    apple_meta = {'released': 2010}
    pear_meta = {'released': 2020, 'details': {'cpu_cores': 4}, 'tags': ['X', 'Y']}
    con.execute(Products.insert(), [
        {'name': 'Apple', 'meta': apple_meta},
        {'name': 'Pear', 'meta': pear_meta},
    ])

In [6]:
# Top-Level members
with con.begin() as t, logs():
    query = (
        Products.select()
        .where(
            Products.c.meta['released'].as_integer()> 2011
        )
    )
    for row in con.execute(query).mappings():
        _meta = row['meta']
        print(row['name'], _meta.get('details', {}).get('cpu_cores'))
        # print('>', row,)

SELECT products.id, products.name, products.meta, products.extra 
FROM products 
WHERE JSON_EXTRACT(products.meta, ?) > ?
[generated in 0.00099s] ('$."released"', 2011)
Pear 4


In [45]:
print(Products.c.meta['tags'].operator)

<function json_getitem_op at 0x0000021BE2DB51C0>


In [11]:
x.as_integer()

In [47]:
# Deeper members 

with con.begin() as t, logs():
    expr = '$."released"'
    expr = '$."details"."cpu_cores"' # works
    expr ='$.details.cpu_cores'  # works
    expr = '$.tags[0]' # works
    expr = '$."tags"[0]' # works
    
    query = (
        Products.select()
        .where(
            # sa.text(f'JSON_EXTRACT(products.meta, \'{expr}\') == "X"')
            # Products.c.meta["details", "cpu_cores"].as_integer() == 4
            sa.func.json_extract(Products.c.meta, expr) == 'X'
            # Products.c.meta[('tags', 1)] == 'X'
        )
    )
    for row in con.execute(query).mappings():
        print('>', row)

SELECT products.id, products.name, products.meta, products.extra 
FROM products 
WHERE json_extract(products.meta, ?) = ?
[cached since 970.1s ago] ('$."tags"[0]', 'X')
> {'id': 2, 'name': 'Pear', 'meta': {'released': 2020, 'details': {'cpu_cores': 4}, 'tags': ['X', 'Y']}, 'extra': None}


In [28]:
with con.begin() as t:
    query = sa.update(Products).values(extra=Products.c['meta'])
    with logs():    
        con.execute(query)
    col = sa.type_coerce(Products.c.extra, sa.JSON)
    query = sa.select(Products).where(col['released'].as_integer() == 2020)
    with logs():
        for row in con.execute(query):
            print(row)
    t.rollback()

UPDATE products SET extra=products.meta
[cached since 362.1s ago] ()
SELECT products.id, products.name, products.meta, products.extra 
FROM products 
WHERE JSON_EXTRACT(products.extra, ?) = ?
[generated in 0.00094s] ('$."released"', 2020)
(2, 'Pear', {'released': 2020}, '{"released": 2020}')


In [7]:
with con.begin() as t:
    query = sa.select(Products.c['meta']['released'])
    for row in con.execute(query).mappings():
        print(row)

{'anon_1': 2010}
{'anon_1': 2020}
